#### @prompt example

## LangChain with GeminiAI
In this notebook we will build two agents for analyzing financial information.
The first agent will use two tools to perform the following functionality#
- Extract the Sector which has performed better over the past week, month, quarter and year

- Once this sector is found , we will ask our agent to retrieve few companies from this sector, and produce a table where those companies are sorted by P/E descending

The second agent will use a Chroma store which has been populated with a document containing Altria Group Q424 earning transcripts, and we  will ask our agent to summarize the call.

#### Installing dependencies

In [1]:
!pip install langchain
!pip install -U langchain-google-genai
!pip install -U -q "google-genai==1.7.0"
!pip install langchain_community
!pip install docx2txt
!pip install chromadb
!pip install wikipedia
!pip install finvizfinance

  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0

#### Loading Keys

In [2]:
import os
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

### Importing Libraries


### Creating our external functions
* get_sectors_performance will retrieve data from finvizfinance containing last week, month, quarter and year peformance. I had to pass a dummy parameter as google API prevented me from using as a tool a function with no parameters. Same function works though with OpenAI LLM
* get companies for sector accepts a sector as input and will retrieve from finvizfinance companies for a specific sector.More filters have been added in the query to make sure that 'enough' data is returned

In [5]:
from langchain_core.tools import tool

@tool
def get_sectors_performance(sector:str = None):
    """ Useful for getting performance of each  sector for last week, last month, last quarter, last half year and last year. **This tool does not require any input from the user.**"""
    
    from finvizfinance.group import Performance

    try:
        performance = Performance()
        # Get the performance data
        return performance.screener_view().to_dict('records')
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure the finvizfinance library is installed correctly.")
        print("You can install it using: pip install finvizfinance")
        print("Also, check your internet connection as the library fetches data from Finviz.")

@tool
def get_companies_for_sector(sectorName:str):
    """ Return a subset of companies for the given sector"""
    from finvizfinance.screener.overview import Overview
    foverview = Overview()
    filters_dict = {'Sector': sectorName,
                    'Market Cap.': '+Small (over $300mln)',
                    'Average Volume': 'Over 200K',
                    'Current Ratio': 'Over 1',
                    'Debt/Equity': 'Under 1',
                    'InstitutionalOwnership': 'Under 60%',
                    'Price': 'Over $10'}
    foverview.set_filter(filters_dict=filters_dict)
    df = foverview.screener_view(order='Company')
    return df.head(5).to_dict('records')


### Chat Memory

In [6]:

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


# Initialize Gemini Pro model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7
)


# Define the tools and create a "tools" node.

tools = [get_sectors_performance, get_companies_for_sector ]

# Attach the tools to the model so that it knows what it can call.
#llm_with_tools = llm.bind_tools(tools)

In [8]:
chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
# Define your prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, max_token_limit=18000)

# Initialize the agent
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

<ipython-input-8-1184b7da5824>:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, max_token_limit=18000)
<ipython-input-8-1184b7da5824>:16: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_chain = initialize_agent(


### Finding the best performing sector across week, month, quarter, and year

In [9]:
input1 = '''Find the sector which across week, month, quarter and year has shown the best performance and summarize it.
'''
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_sectors_performance
Action Input: 
Observation: [{'Name': 'Basic Materials', 'Perf Week': '6.66%', 'Perf Month': -0.030299999999999997, 'Perf Quart': 0.0096, 'Perf Half': -0.1114, 'Perf Year': -0.1001, 'Perf YTD': 0.0102, 'Recom': 1.99, 'Avg Volume': 695860000.0, 'Rel Volume': 2.19, 'Change': 0.0364, 'Volume': 1520000000.0}, {'Name': 'Communication Services', 'Perf Week': '5.61%', 'Perf Month': -0.0392, 'Perf Quart': -0.0809, 'Perf Half': -0.0055000000000000005, 'Perf Year': 0.0728, 'Perf YTD': -0.077, 'Recom': 1.62, 'Avg Volume': 723960000.0, 'Rel Volume': 0.88, 'Change': 0.0111, 'Volume': 639070000.0}, {'Name': 'Consumer Cyclical', 'Perf Week': '3.50%', 'Perf Month': -0.0404, 'Perf Quart': -0.1378, 'Perf Half': -0.06709999999999999, 'Perf Year': 0.0084, 'Perf YTD': -0.1501, 'Recom': 1.8, 'Avg Volume': 1680000000.0, 'Rel Volume': 1.58, 'Change': 0.013000000000000001, 'Volume': 2650000000.0}, {'Na

### Now we are going to find few companies for the best performing sector. We'll output a table sorted by P/E

In [10]:
input1 = "Now find me some companies for the sector you found in the previous step. Create  a table with Ticker, Company,  P/E and change"
result = agent_chain.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: get_companies_for_sector
Action Input: Utilities
Observation: [{'Ticker': 'CEPU', 'Company': 'Central Puerto ADR', 'Sector': 'Utilities', 'Industry': 'Utilities - Regulated Electric', 'Country': 'Argentina', 'Market Cap': 1620000000.0, 'P/E': 32.86, 'Price': 10.7, 'Change': 0.08410000000000001, 'Volume': 397671.0}, {'Ticker': 'OKLO', 'Company': 'Oklo Inc', 'Sector': 'Utilities', 'Industry': 'Utilities - Regulated Electric', 'Country': 'USA', 'Market Cap': 3130000000.0, 'P/E': nan, 'Price': 22.5, 'Change': -0.0097, 'Volume': 7715040.0}, {'Ticker': 'PAM', 'Company': 'Pampa Energia SA ADR', 'Sector': 'Utilities', 'Industry': 'Utilities - Independent Power Producers', 'Country': 'Argentina', 'Market Cap': 3780000000.0, 'P/E': 6.01, 'Price': 69.59, 'Change': 0.0696, 'Volume': 208763.0}]
Thought:Do I need to use a tool? No
AI: Here are some companies in the Utilities sector, based on the previous analys

### Now we will play around with Chroma
### We will upload an extract from Altria Group latest Earning calls and ask our llm to extract some information

In [83]:
!pip install -U -q "google-genai==1.7.0"

In [22]:
from langchain.agents import initialize_agent, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

import os

# --- Configure Google API Key (if needed) ---
# os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY"

# --- Define the LLM and Embeddings ---
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# --- Load and Index the Text Document ---
def load_document_into_chroma(file_path, persist_directory="chroma_db_single"):
    """Loads a text document, creates embeddings, and stores it in ChromaDB."""
    loader = TextLoader(file_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(documents)
    vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
    vectorstore.persist()
    return vectorstore

# --- Define the Retrieval Tool ---
def query_chroma_document(query):
    """Queries the ChromaDB for the loaded document."""
    vectorstore = Chroma(persist_directory="chroma_db_single", embedding_function=embeddings)
    retriever = vectorstore.as_retriever()
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

vectorstore = load_document_into_chroma('/kaggle/input/altria-q424-earning-call/Altria_Q424EarningCall.txt')

retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs = {'k' : 3, 'lambda_mult' : 0.7})
retriever_tool = create_retriever_tool(retriever= retriever,
                                        name="Altria Q424 Earning calls",
                                        description="For any questions regarding Altria latest earning call you must use this tool")

# --- Create the Retrieval Tool ---
retrieval_tool = Tool(
    name="Document Retriever",
    func=query_chroma_document,
    description="Useful for retrieving specific content from the loaded document.",
)

# --- Define the Prompt Template for the Agent ---
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that can retrieve information from a specific document based on user queries.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        ("user", "Use the 'Document Retriever' tool to find relevant information in the document."),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# --- Initialize Memory for the Agent ---
memory = ConversationBufferMemory(memory_key=MEMORY_KEY, return_messages=True)

# --- Initialize the Agent ---
chromaagent = initialize_agent(
    [retrieval_tool],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    prompt=prompt,
)



### Now we'll query to find highlights from Altria latest earning calls

In [23]:
input1 = '''Please provide the main higlights from latest earning call transcript'''
result = chromaagent.invoke({"input": input1})
print(result['output'])



> Entering new AgentExecutor chain...
I need to retrieve the highlights from the latest earnings call transcript.
Action: Document Retriever
Action Input: "highlights from latest earnings call transcript"
Observation: quarter, down 8.1% versus the prior year. These earnings reflect the impact of a lower ownership interest compared to the year-ago period due to the partial sale of our ABI investment last year. We continue to view the ABI stake as a financial investment and our goal remains to maximize the long-term value of the investment for our shareholders. We demonstrated our commitment to returning significant value to shareholders and maintaining a strong balance sheet during 2024. For the full year, we returned over $10.2 billion of cash to shareholders through dividends and share repurchases. We paid $6.8 billion in dividends and our board raised our dividend by 4.1% in August, marking our 59th increase in the last 55 years. We also completed our previously authorized share re

<ipython-input-22-d325d5e66e8f>:34: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="chroma_db_single", embedding_function=embeddings)
<ipython-input-22-d325d5e66e8f>:36: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


The highlights from the latest earnings call transcript are:

*   Earnings were down 8.1% versus the prior year, reflecting the impact of a lower ownership interest due to the partial sale of their ABI investment.
*   The company views the ABI stake as a financial investment and aims to maximize its long-term value.
*   In 2024, the company returned over $10.2 billion of cash to shareholders through dividends and share repurchases.
*   They paid $6.8 billion in dividends, and the board raised the dividend by 4.1% in August, marking the 59th increase in the last 55 years.
*   The company completed its share repurchase program, repurchasing 73.5 million shares for $3.4 billion, the largest single-year repurchase in over two decades.
*   Share repurchases were supported by proceeds from the partial sale of ABI investment.

Final Answer: The highlights from the latest earnings call transcript are:

*   Earnings were down 8.1% versus the prior year, reflecting the impact of a lower ownershi